In [1]:
import cv2
import os
import random
import mediapipe as mp

In [15]:
# global variables
rad = 5
radiusImcrement = 3 # 8 for development purpose 3 for final
pos = [100,100]
OnOff = 1 # -1 means ball is not drawing and 1 means ball is drawing
timer = 0
timer1 = 0
stumpsTopLeft = (300,300);
stumpsBottomRight = (380,480);

In [24]:
def drawBall(img,centerTuple,radi)->None:
    cv2.circle(img,centerTuple,radi,(0,0,255),cv2.FILLED)

def generateRandomCordinates():
    return (random.randint(0,500),random.randint(0,450))

def reset():
    pos[0],pos[1] = generateRandomCordinates()
    pass

def drawStumps(img, topLeft, bottomRight):
    cv2.rectangle(img,topLeft,bottomRight, (0,255,0), 5)

def checkInStumps(centreTuple, onOff, topLeft, bottomRight):
    if onOff == 1:
        if centreTuple[0] > topLeft[0] and centreTuple[0] < bottomRight[0]:
            if centreTuple[1] > topLeft[1] and centreTuple[1] < bottomRight[1]:
                return True
    return False

    

In [4]:
hands = mp.solutions.hands
hand = hands.Hands(min_detection_confidence=0.1, min_tracking_confidence=0.1)
drawFunc = mp.solutions.drawing_utils



HandLandmarks I want are 5,13,9,17
PoseLandmarks I want are 11,13,12,14


In [5]:
def seeHandLandmarks(vals : list [int],img,resultsHand):
    for i in vals:
        if resultsHand.multi_hand_landmarks and i < len(resultsHand.multi_hand_landmarks):
            drawFunc.draw_landmarks(img, resultsHand.multi_hand_landmarks[i], mp.solutions.hands.HAND_CONNECTIONS)
            



In [25]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert the image from BGR to RGB
    resultsHand = hand.process(imgRGB) # process the image
    if resultsHand.multi_hand_landmarks:
        for handLms in resultsHand.multi_hand_landmarks:
            drawFunc.draw_landmarks(img, handLms, mp.solutions.hands.HAND_CONNECTIONS)
    rad = rad + radiusImcrement
    if OnOff==1:
        drawBall(img,(pos[0],pos[1]),int(rad))
    if rad>35:
        radiusImcrement = -1*(radiusImcrement)
    if rad<5:
        radiusImcrement = -1*(radiusImcrement)
        OnOff = (-1)*(OnOff)
        reset()

    drawStumps(img,stumpsTopLeft,stumpsBottomRight)
    
    # print(checkInStumps((pos[0],pos[1]),OnOff,stumpsTopLeft,stumpsBottomRight))
    if checkInStumps((pos[0],pos[1]),OnOff,stumpsTopLeft,stumpsBottomRight):
        print(True)
    cv2.imshow("Video", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
